<a href="https://colab.research.google.com/github/gagan3012/gsoc-ml/blob/master/Test_II_Learning_the_DM_representation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>